In [1]:
import os
import sys
import math
import logging
import structlog
from pathlib import Path
import json

import tomli
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

In [2]:
logging.basicConfig(level=logging.WARNING, stream=sys.stdout)

In [3]:
import pytanis
from pytanis import GSheetClient, PretalxClient, HelpDeskClient
from pytanis.review import Col
from pytanis.helpdesk import Mail, Recipient, MailClient

In [4]:
# Be aware that this notebook might only run with the following version
pytanis.__version__ 

'0.4'

In [5]:
# Import event-specific settings to don't have them here in the notebook
with open('config.toml', 'rb') as fh:
    cfg = tomli.load(fh)
    
# CAREFUL: Setting this to false will send e-mails to a lot of people!
DRY_RUN = True

# Get all the Reviewers

In [6]:
gsheet_client = GSheetClient()
gsheet_df = gsheet_client.gsheet_as_df(cfg['spreadsheet_id'], cfg['worksheet_name'])
# rename columns to stick to our convention
col_map = {
 "Topics you want to review": Col.track_prefs,
 "Email address": Col.email,
 "Name": Col.speaker_name,
 "Affiliation": Col.affiliation,
 "Who do you know from the Committee?": Col.committee_contact,
 "Availability during the Review Period": Col.availability,
 "Additional comments regarding your availability during the review period.": Col.availability_comment,
 "Activated in Pretalx": Col.pretalx_activated,
 "Do you want your name to be listed as a reviewer on the conference website?": Col.public,
 "Wants all proposals": Col.all_proposals,
 "Any additional comments for the Program Committee": Col.comment,
}
gsheet_df.rename(columns=col_map, inplace=True)

In [7]:
# add column to address people nicely
gsheet_df[Col.address_as] = gsheet_df[Col.speaker_name].apply(lambda x: x.split()[0].title())

In [8]:
# determine reviewers having not even activated the Pretalx Acccount and mark them as recipients
reviewers_not_activated = gsheet_df.loc[gsheet_df[Col.pretalx_activated].isna(), [Col.speaker_name, Col.email, Col.address_as]]
reviewers_not_activated = reviewers_not_activated.apply(lambda x: Recipient(name=x[Col.speaker_name], email=x[Col.email], address_as=x[Col.address_as]), axis=1).to_list()

# Mail to Reviewers that haven't activated their account in Pretalx

In [10]:
mail_body = """
Howdy {recipient.address_as}!

The review of proposals for the PyConDE / PyData is already in full swing and we are missing you!

It seems you haven't yet activated your Pretalx Account for reviewing. Please check you e-mails (and also spam)
if you have received an invitation to a special reviewing team. Please click the link in the e-mail and hit also the "accept" button on the emerging page.
We just sent out a new re-invitations e-mail to your e-mail address. If you need help with the activation, please contact program23@pycon.de, we are happy to help you.
After you have activated, we will start assigning proposals to you, so please check back frequently.

Information from our past e-mails:
* Reviewer Guidelines (https://bit.ly/pyconde23-reviewer-guidelines)
* [Nonobligatory] 2nd live “Meet and Greet” session: 17 January, 17:00 (CET) in Gather Town (https://bit.ly/pyconde23-meet-reviewers)
* Review to be finished by: 31 January, 00:00 (CET)

We are looking forward to see you {recipient.address_as} and thank you very much for your support!


All the best,
Program Committee
PyCon DE & PyData Berlin 2023
"""

In [11]:
mail = Mail(
    subject="Please activate your Pretalx Account to review PyCon DE / PyData proposals",
    text=mail_body,
    team_id=cfg["team_id"],
    agent_id=cfg["agent_id"],
    status="solved",
    recipients=reviewers_not_activated
)

In [ ]:
mail_client = MailClient()
responses, errors = mail_client.send(mail, dry_run=DRY_RUN)
assert not errors

# Mail to Reviewers activated in Pretalx

In [13]:
activated_reviewers = gsheet_df.loc[~gsheet_df[Col.pretalx_activated].isna()]

### Analyse the current reviews and determine top X% reviewers

In [14]:
pretalx_client = PretalxClient()
n_reviews, reviews = pretalx_client.reviews(cfg['event_name'])
reviews = list(reviews)

In [15]:
scored_reviews_df = pd.DataFrame([{"user": r.user, "score": r.score, "n_reviews": r.submission} for r in reviews if r.score is not None])
scored_reviews_df = scored_reviews_df.groupby("user").count()[["n_reviews"]]
scored_reviews_df['top_perc'] =  (1. - scored_reviews_df.rank(pct=True)["n_reviews"])

def top_perc_text(perc):
    if perc <= 0.1:
        return "top 10%"
    elif perc <= 0.25: 
        return "top 25%"
    elif perc <= 0.5: 
        return "top 50%"
    elif perc <= 0.75: 
        return "top 75%"
    elif perc <= 0.90: 
        return "top 90%"
    else:
        return "top 100% ;-)"

scored_reviews_df['top_perc_text'] = scored_reviews_df['top_perc'].apply(top_perc_text)

### Merge back with all reviewers and generate a nice feedback message

In [16]:
activated_reviewers = pd.merge(activated_reviewers, scored_reviews_df, right_on='user', left_on='Pretalx Name', how='left')
activated_reviewers["n_reviews"].fillna(0., inplace=True)

In [17]:
def get_feedback(x):
    if x['n_reviews'] == 0.:
        return "So far you haven't reviewed any proposals, it's time to get started!"
    else:
        return f"Thanks for having already started to reviews! You are a champion and currently in the {x['top_perc_text']} of all reviewers!"

activated_reviewers["feedback"] = activated_reviewers.apply(get_feedback, axis=1)

### Create and send an individual e-mail to each active reviewer

In [18]:
active_recipients = activated_reviewers.apply(lambda x: Recipient(name=x[Col.speaker_name], 
                                                                  email=x[Col.email], 
                                                                  address_as=x[Col.address_as], 
                                                                  data={"feedback": x["feedback"]}),  axis=1).to_list()

In [19]:
mail_body = """
Hi {recipient.address_as}!

The review of proposals for the PyConDE / PyData is already in full swing and we are happy to have you on board!
{recipient.data.feedback}

Please check back frequently into Pretalx as we will soon assign proposals more dynamically. 
PyConDE / PyData is completely community driven by volunteers like you and we highly appreciate your support!
We will keep you updated.

Information from our past e-mails:
* Reviewer Guidelines (https://bit.ly/pyconde23-reviewer-guidelines)
* [Nonobligatory] 2nd live “Meet and Greet” session: 17 January, 17:00 (CET) in Gather Town (https://bit.ly/pyconde23-meet-reviewers)
* Review to be finished by: 31 January, 00:00 (CET)
* Contact program23@pycon.de for support if needed

Thank you very much {recipient.address_as} for your support!


All the best,
Program Committee
PyCon DE & PyData Berlin 2023
"""

In [20]:
mail = Mail(
    subject="Update for you on the review process for PyCon DE / PyData!",
    text=mail_body,
    team_id=cfg["team_id"],
    agent_id=cfg["agent_id"],
    status="solved",
    recipients=active_recipients
)

In [ ]:
responses, errors = mail_client.send(mail, dry_run=DRY_RUN)
assert not errors

## Reminder e-mail to everyone activated

In [22]:
def get_feedback(x):
    if x['n_reviews'] == 0.:
        return "So far you haven't reviewed any proposals, it seems. Now it's really time to get started :-)\nPlease let us know if you are not able to review for some reason. In this case, we must assign your proposals to others soon."
    else:
        return f"Thanks that you already supported us so much! We are close to the finish line. Go for it, champion!"

activated_reviewers["feedback"] = activated_reviewers.apply(get_feedback, axis=1)

In [23]:
active_recipients = activated_reviewers.apply(lambda x: Recipient(name=x[Col.speaker_name], 
                                                                  email=x[Col.email], 
                                                                  address_as=x[Col.address_as], 
                                                                  data={"feedback": x["feedback"]}),  axis=1).to_list()

In [24]:
mail_body = """
Hi {recipient.address_as}!

We hope you had a great start into the new week :-)
There are only 8 days left until the deadline of our review process on 31 January, 00:00 (CET). 
We have reached now 70% of all the reviews we need, that's awesome! But still a few miles to go for us.
{recipient.data.feedback}

Please check back frequently into Pretalx as there might be only a few more proposals in case reviewers dropped out.
PyConDE / PyData is completely community driven by volunteers like you and we highly appreciate your support!
We will keep you updated.

Information from our past e-mails:
* Reviewer Guidelines (https://bit.ly/pyconde23-reviewer-guidelines)
* Contact program23@pycon.de for support if needed

Thank you very much {recipient.address_as} for your support!


All the best,
Program Committee
PyCon DE & PyData Berlin 2023
"""

In [25]:
mail = Mail(
    subject="Reminder: deadline of the review process for PyCon DE / PyData is coming closer!",
    text=mail_body,
    team_id=cfg["team_id"],
    agent_id=cfg["agent_id"],
    status="solved",
    recipients=active_recipients
)

In [ ]:
responses, errors = mail_client.send(mail, dry_run=DRY_RUN)
assert not errors